### 0. Imports and requirements

* Для моделирования нам понадобится реализация градиентного бустинга из `lightgbm`. Для обработки данных &ndash; стандартный DS стек из `pandas`, `numpy` и `sklearn`. Так как мы будем использовать достаточно объемные выборки данных (кредитные истории клиентов), то нужно будет эффективно читать данные и выделять из них признаки, чтобы иметь возможность строить решение даже на локальных машинах с ограничениями по оперативной памяти.  Поэтому нам понадобятся инструменты для работы с форматом данных `Parquet`. Библиотека `pandas` представляет необходимый инструментарий (рекомендуем установить модуль `fastparquet`, это позволит еще быстрее считывать данные).

In [1]:
%load_ext autoreload
%autoreload 2

import gc
import os
import sys
import pandas as pd
import numpy as np
import tqdm
import seaborn as sns


import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score

import catboost as cb
import lightgbm as lgb

pd.set_option('display.max_columns', None)

# если у вас есть CUDA, то она понадобится там для экспериментов в catboost
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

sys.path.append('../')

### 1. Reading Data

* В данном соревновании участникам предстоит работать с таким форматом данных, как `Parquet`. Узнать подробнее об этом формате Вы можете самостоятельно. Самое главное &ndash; это крайне эффективный бинарный формат сжатия данных по колонокам. Однако, для непосредственной работы с данными и построения моделей нам нужно прочитать их и трансформировать в pandas.DataFrame. При этом сделать это эффективно по памяти.

In [2]:
TRAIN_DATA_PATH = "../data/train_data/"
TEST_DATA_PATH = "../data/test_data/"

TRAIN_TARGET_PATH = "../data/train_target.csv"

In [3]:
from utils import read_parquet_dataset_from_local

* Для примера прочитаем одну партицию в память и оценим, сколько RAM занимает весь DataFrame

In [4]:
data_frame = read_parquet_dataset_from_local(TRAIN_DATA_PATH, start_from=0, num_parts_to_read=1)

memory_usage_of_frame = data_frame.memory_usage(index=True).sum() / 10**9
expected_memory_usage = memory_usage_of_frame * 12
print(f"Объем памяти в RAM одной партиции данных с кредитными историями: {round(memory_usage_of_frame, 3)} GB")
print(f"Ожидаемый размер в RAM всего датасета: {round(expected_memory_usage, 3)} GB")

Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Объем памяти в RAM одной партиции данных с кредитными историями: 0.964 GB
Ожидаемый размер в RAM всего датасета: 11.564 GB


* Итого, при чтении всех данных сразу, они займут значительный объем памяти. Решение &ndash; читать данные итеративно небольшими чанками. Чанки организованы таким образом, что для конкретного клиента вся информация о его кредитной истории до момента подачи заявки на кредит расположена внутри одного чанка. Это позволяет загружать данные в память небольшими порциями, выделять все необходимые признаки и получать результирующий фрейм для моделирования. Для этих целей мы предоставляем вам функцию `utils.read_parquet_dataset_from_local`.

Так же все чанки данных упорядочены по возрастанию даты заявки на кредит.

In [5]:
del data_frame
gc.collect()

43

### 2. Feature Extraction

* В рамках данного соревнования участникам предстоит работать с кредитной историей клиентов. На основе кредитной истории клиента до момента подачи заявки на новый кредит нужно оценить, насколько благонадежным является клиент, и определить вероятность его ухода в дефолт по новому кредиту. Каждый кредит описывается набором из 60 категориальных признаков.


* Базовым подходом к решению этой задачи является построение классических моделей машинного обучения на аггрегациях от последовательностей категориальных признаков. В данном случае мы закодируем признаки с помощью one-hot-encoding'а, применим к ним count-аггрегирование и обучим на этом градиентный бустинг из реализации `lightgbm`.
     
     
 * Описанный подход к обработке кредитных историй клиентов реализован в виде класс-трансформера `CountAggregator` ниже:

In [6]:
TRAIN_FEATURES_PATH = "../data/train_features_gb/"
TEST_FEATURES_PATH = "../data/test_features_gb/"

In [7]:
! rm -r TRAIN_FEATURES_PATH
! mkdir TRAIN_FEATURES_PATH
! rm -r TEST_FEATURES_PATH
! mkdir TEST_FEATURES_PATH

In [8]:
class CountAggregator(object):
    
    def __init__(self):
        self.encoded_features = None
        
    def __extract_count_aggregations(self, data_frame: pd.DataFrame, mode: str) -> pd.DataFrame:
        # one-hot-encoding
        feature_columns = list(data_frame.columns.values)
        feature_columns.remove("id")
        feature_columns.remove("rn")

        dummies = pd.get_dummies(data_frame[feature_columns], columns=feature_columns)
        dummy_features = dummies.columns.values
        
        ohe_features = pd.concat([data_frame, dummies], axis=1)
        ohe_features = ohe_features.drop(columns=feature_columns)
        
        # count aggregation
        ohe_features.groupby("id")
        features = ohe_features.groupby("id")[dummy_features].sum().reset_index(drop=False)
        return features
        
    def __transform_data(self, path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int=50,
                                     mode: str = "fit_transform", save_to_path=None, verbose: bool=False):
        assert mode in ["fit_transform", "transform"], f"Unrecognized mode: {mode}! Please use one of the following modes: \"fit_transform\", \"transform\""
        preprocessed_frames = []
        for step in tqdm.tqdm_notebook(range(0, num_parts_total, num_parts_to_preprocess_at_once), 
                                       desc="Transforming sequential credit data"):
            data_frame = read_parquet_dataset_from_local(path_to_dataset, start_from=step, 
                                                         num_parts_to_read=num_parts_to_preprocess_at_once, 
                                                         verbose=verbose)
            features = self.__extract_count_aggregations(data_frame, mode=mode)
            if save_to_path:
                features.to_parquet(os.path.join(save_to_path, f"processed_chunk_{step}.pq"))
            preprocessed_frames.append(features)
        
        features = pd.concat(preprocessed_frames)
        features.fillna(np.uint8(0), inplace=True)
        dummy_features = list(features.columns.values)
        dummy_features.remove("id")
        if mode == "fit_transform":
            self.encoded_features = dummy_features
        else:
            assert not self.encoded_features is None, "Transformer not fitted"
            for col in self.encoded_features:
                if not col in dummy_features:
                    features[col] = np.uint8(0)
        return features[["id"]+self.encoded_features]
    
    def fit_transform(self, path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int = 50,
                      save_to_path=None, verbose: bool=False):
        return self.__transform_data(path_to_dataset=path_to_dataset,
                                     num_parts_to_preprocess_at_once=num_parts_to_preprocess_at_once,
                                     num_parts_total=num_parts_total, mode="fit_transform",
                                     save_to_path=save_to_path, verbose=verbose)
    def transform(self, path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int=50,
                  save_to_path=None, verbose: bool=False):
        return self.__transform_data(path_to_dataset=path_to_dataset,
                                     num_parts_to_preprocess_at_once=num_parts_to_preprocess_at_once,
                                     num_parts_total=num_parts_total, mode="transform",
                                     save_to_path=save_to_path, verbose=verbose)

In [9]:
%%time
aggregator = CountAggregator()
train_data = aggregator.fit_transform(TRAIN_DATA_PATH, num_parts_to_preprocess_at_once=4, num_parts_total=12, 
                                      save_to_path=TRAIN_FEATURES_PATH, verbose=True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Transforming sequential credit data:   0%|          | 0/3 [00:00<?, ?it/s]

Reading chunks:
../data/train_data/train_data_0.pq
../data/train_data/train_data_1.pq
../data/train_data/train_data_2.pq
../data/train_data/train_data_3.pq


Reading dataset with pandas:   0%|          | 0/4 [00:00<?, ?it/s]

Reading chunks:
../data/train_data/train_data_4.pq
../data/train_data/train_data_5.pq
../data/train_data/train_data_6.pq
../data/train_data/train_data_7.pq


Reading dataset with pandas:   0%|          | 0/4 [00:00<?, ?it/s]

Reading chunks:
../data/train_data/train_data_8.pq
../data/train_data/train_data_9.pq
../data/train_data/train_data_10.pq
../data/train_data/train_data_11.pq


Reading dataset with pandas:   0%|          | 0/4 [00:00<?, ?it/s]

CPU times: user 5min 46s, sys: 5min 18s, total: 11min 5s
Wall time: 10min 5s


In [10]:
train_data.isna().sum().sum()

0

In [11]:
print(f"Объем оперативной памяти, занимаемой датафреймом с признаками: {train_data.memory_usage(index=True).sum() / 1e9}")

Объем оперативной памяти, занимаемой датафреймом с признаками: 1.683


* Также извлечем признаки для заявок из тестовой выборки

In [12]:
test_data = aggregator.transform(TEST_DATA_PATH, num_parts_to_preprocess_at_once=2, num_parts_total=2,
                                 save_to_path=TEST_FEATURES_PATH, verbose=True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Transforming sequential credit data:   0%|          | 0/1 [00:00<?, ?it/s]

Reading chunks:
../data/test_data/test_data_0.pq
../data/test_data/test_data_1.pq


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

### 3. Train Model: LightGBM + CV

* Для обучения модели помимо признаковых описаний нам также нужна целевая переменная. Целевая переменная для тренировочной выборки содержится в файле train_target.csv.

In [13]:
TRAIN_TARGET_PATH = "../data/train_target.csv"

In [14]:
train_target = pd.read_csv(TRAIN_TARGET_PATH)

In [15]:
train_data_target = train_target.merge(train_data, on="id")

In [16]:
train_data_target.shape

(3000000, 421)

In [17]:
feature_cols = list(train_data_target.columns.values)
feature_cols.remove("id"), feature_cols.remove("flag")
len(feature_cols)

419

In [18]:
targets = train_data_target["flag"].values

cv = KFold(n_splits=5, random_state=100, shuffle=True)

oof = np.zeros(len(train_data_target))
train_preds = np.zeros(len(train_data_target))

models = []

tree_params = {
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.05,
    "max_depth": 5,
    "reg_lambda": 1,
    "num_leaves": 64,
    "n_jobs": 5,
    "n_estimators": 2000
}

for fold_, (train_idx, val_idx) in enumerate(cv.split(train_data_target, targets), 1):
    print(f"Training with fold {fold_} started")
    lgb_model = lgb.LGBMClassifier(**tree_params)
    train, val = train_data_target.iloc[train_idx], train_data_target.iloc[val_idx]
    
    lgb_model.fit(train[feature_cols], train.flag.values, eval_set=[(val[feature_cols], val.flag.values)],
              early_stopping_rounds=50, verbose=50)

    oof[val_idx] = lgb_model.predict_proba(val[feature_cols])[:, 1]
    train_preds[train_idx] += lgb_model.predict_proba(train[feature_cols])[:, 1] / (cv.n_splits-1)
    models.append(lgb_model)
    print(f"Training with fold {fold_} completed")

Training with fold 1 started
Training until validation scores don't improve for 50 rounds
[50]	valid_0's auc: 0.738614
[100]	valid_0's auc: 0.747327
[150]	valid_0's auc: 0.751475
[200]	valid_0's auc: 0.753816
[250]	valid_0's auc: 0.755322
[300]	valid_0's auc: 0.756596
[350]	valid_0's auc: 0.757394
[400]	valid_0's auc: 0.758024
[450]	valid_0's auc: 0.75856
[500]	valid_0's auc: 0.759029
[550]	valid_0's auc: 0.759293
[600]	valid_0's auc: 0.759525
[650]	valid_0's auc: 0.759801
[700]	valid_0's auc: 0.760127
[750]	valid_0's auc: 0.760259
[800]	valid_0's auc: 0.760416
[850]	valid_0's auc: 0.76063
[900]	valid_0's auc: 0.760767
[950]	valid_0's auc: 0.760866
[1000]	valid_0's auc: 0.761047
[1050]	valid_0's auc: 0.761213
[1100]	valid_0's auc: 0.761264
[1150]	valid_0's auc: 0.761389
[1200]	valid_0's auc: 0.761439
[1250]	valid_0's auc: 0.761575
[1300]	valid_0's auc: 0.761673
[1350]	valid_0's auc: 0.761724
[1400]	valid_0's auc: 0.761776
[1450]	valid_0's auc: 0.761836
[1500]	valid_0's auc: 0.761903
[1

In [19]:
print("Train roc-auc: ", roc_auc_score(targets, train_preds))

Train roc-auc:  0.8031990015398422


In [20]:
print("CV roc-auc: ", roc_auc_score(targets, oof))

CV roc-auc:  0.763063554599203


### 4. Submission

* Подготовим посылку в проверяющую систему

In [21]:
score = np.zeros(len(test_data))

for model in tqdm.tqdm_notebook(models):
    score += model.predict_proba(test_data[feature_cols])[:, 1] / len(models)
    
submission = pd.DataFrame({
    "id" : test_data["id"].values,
    "score": score
}) 

submission.to_csv("submission.csv", index=None) # ~ 0.795 roc-auc на public test

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/5 [00:00<?, ?it/s]